# KNN

# Imports

In [1]:
!pip install pandas==1.4.2
!pip install matplotlib==3.5.2
!pip install seaborn==0.11.2
!pip install nltk==3.7
!pip install scikit-learn==1.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.4.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
import pandas as pd

In [3]:
# Change the file directories to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_train_valid = pd.read_parquet('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/train')
df_test = pd.read_parquet('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/test')

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

In [5]:
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sklearn.impute import SimpleImputer

In [7]:
import numpy as np

In [8]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [9]:
from sklearn.model_selection import RandomizedSearchCV

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
df_train_valid.surprise1.value_counts()

the world       24893
business         4649
sports           4103
science/tech     2018
Name: surprise1, dtype: int64

# Agrego valores a una columna vacía

In [12]:
df_train_valid['content'] = df_train_valid['content'].fillna(value='')
df_test['content'] = df_test['content'].fillna(value='')

In [13]:
count_vectorizer = CountVectorizer(lowercase=True, max_features=5000, stop_words='english')

array = count_vectorizer.fit_transform(df_train_valid['content'].tolist()).toarray()
df_train_valid['n_non_stop_words'] = [sum(arr) for arr in array]

array = count_vectorizer.transform(df_test['content'].tolist()).toarray()
df_test['n_non_stop_words'] = [sum(arr) for arr in array]

# Separo el dataframe de train en: train y valid

In [14]:
#Ordeno el df de train_valid
df_train_valid = df_train_valid.sort_values('timedelta', ascending=False)
#Separo el df de train en train y validation
df_train = df_train_valid.iloc[:89*(len(df_train_valid)//100)].copy()
df_valid = df_train_valid.iloc[89*(len(df_train_valid)//100):].copy()

# Encoding

Encodeo surprise2 con MeanEncoding

In [15]:
df_train['surprise2'] =  df_train['surprise2'].map(df_train.groupby(['surprise2'])['popular'].mean().to_dict())
df_valid['surprise2'] =  df_valid['surprise2'].map(df_valid.groupby(['surprise2'])['popular'].mean().to_dict())
df_test['surprise2'] =  df_test['surprise2'].map(df_test.groupby(['surprise2'])['popular'].mean().to_dict())

Encodeo surprise1 con OneHotEncoder

In [16]:
ohe_surp1 = OneHotEncoder()

surp1_train = ohe_surp1.fit_transform(df_train['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)
surp1_valid = ohe_surp1.transform(df_valid['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)
surp1_test = ohe_surp1.transform(df_test['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)

df_train[list(ohe_surp1.categories_[0])] = surp1_train
df_valid[list(ohe_surp1.categories_[0])] = surp1_valid
df_test[list(ohe_surp1.categories_[0])] = surp1_test

Encodeo content con CountVectorizer

In [17]:
count_vectorizer = CountVectorizer(lowercase=True, max_features=10, stop_words='english')

In [18]:
dataframes = [df_train, df_valid, df_test]

In [19]:
for index, df in enumerate(dataframes):

  if index == 0:
    array = count_vectorizer.fit_transform(df['content'].tolist()).toarray()
  else:
    array = count_vectorizer.transform(df['content'].tolist()).toarray()

  names = count_vectorizer.get_feature_names_out()

  for index, name in enumerate(names):
    df[name] = [arr[index] for arr in array]

# Separo dfs en x e y

In [20]:
#Dropeo los features que no me sirven
x_train_df = df_train.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_train = df_train['popular']

x_valid_df = df_valid.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_valid = df_valid['popular']

x_test_df = df_test.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_test = df_test['popular']

# Reemplazo los NaN

In [21]:
imputer = SimpleImputer(strategy='mean', copy=False)

x_train = imputer.fit_transform(x_train_df)
x_valid = imputer.transform(x_valid_df)
x_test = imputer.transform(x_test_df)

# Normalizo los features

In [22]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

# Modelo base

In [23]:
model = KNeighborsClassifier()

In [24]:
model.fit(x_train, y_train)

KNeighborsClassifier()

# Modelo con mejores hiperparámetros

In [31]:
model = KNeighborsClassifier(algorithm='ball_tree', leaf_size=10, n_neighbors=100)

In [32]:
model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=10, n_neighbors=100)

# Modelo con búsqueda de hiperparámetros

In [27]:
model = KNeighborsClassifier()

In [28]:
params = {
    'n_neighbors': [50, 80, 100, 150],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree'],
    'leaf_size': [5, 10, 25, 40, 50]
}

In [29]:
model = RandomizedSearchCV(model, params, n_iter=25, random_state=10)

In [30]:
model.fit(x_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
model = model.best_estimator_

In [ ]:
model

# Resultados

In [33]:
binary_preds = model.predict(x_valid)
proba_preds = model.predict_proba(x_valid)[:,1]

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [34]:
print(classification_report(y_valid, binary_preds))

              precision    recall  f1-score   support

       False       0.83      1.00      0.91      3291
        True       0.00      0.00      0.00       688

    accuracy                           0.83      3979
   macro avg       0.41      0.50      0.45      3979
weighted avg       0.68      0.83      0.75      3979



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
roc_auc_score(y_valid, proba_preds)

0.6618303177093271